In [1]:
import os
import pandas as pd

In [2]:
cd ..

/home/evan/Documents/github/Feedlot


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

### load transformed chainlink price data from data/transformed/chainlink_prices folder

In [4]:
# get name of each file in chainlink_prices folder without the .csv
files = [f.split('.')[0] for f in os.listdir('data/transformed/chainlink_prices')]

# sort files alphabetically
files.sort()

### load cow_df dataset from data/transformed folder

In [5]:
# load cow swaps dataset
cow_df = pd.read_csv('data/transformed/cow_swaps.csv', low_memory=False)

In [6]:
# sort cow_df by timestamp
cow_df = cow_df.sort_values(by='timestamp')

In [7]:
# add new date column based off of timestamp conversion
cow_df['date'] = pd.to_datetime(cow_df['timestamp'], unit='s')

In [8]:
cow_df.head(5)

,timestamp,gasPrice,feeAmount,txHash,settlement,buyAmount,sellAmount,order,buyAmountEth,sellAmountEth,buyAmountUsd,sellAmountUsd,solvers,environment,name,active,sellTokenSymbol,sellTokenDecimals,buyTokenSymbol,buyTokenDecimals,date
177087,1628089484,55000000474,6776000000000000,0x08100e7ba81be84ee0bdce43db6640e2f992ec9991a740a689e97d20dea9dafa,0x08100e7ba81be84ee0bdce43db6640e2f992ec9991a740a689e97d20dea9dafa,1225092772597923267,100000000000000000,0x81dbb9631c0bec7d9762e170ff5feb2c9b366d5ca9502036d20d99c91cafb8bfb54872859733a3dfbb2c5401ac68cd9ca84b3cd1610ab36f,0.093887,0.100000,246.157990,264.102661,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,WETH,18,GNO,18,2021-08-04 15:04:44
44069,1628146744,44000000000,3663931816677019648,0x8772387c9e80b23080cba1ce607defc91f6680d5803f745dedbcc218e98562e8,0x8772387c9e80b23080cba1ce607defc91f6680d5803f745dedbcc218e98562e8,17868105041905652,17060511063645525663,0xee0db370db55b657a0a2e507249d6e17575196c82434dee380fc1922e94b37967b2e78d4dfaaba045a167a70da285e30e8fca196610b92dc,0.017868,0.022882,47.899028,61.470674,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,BNT,18,WETH,18,2021-08-05 06:59:04
148297,1628150635,53250000505,5000000000000000,0x5baf36801788b2d23d99b0797cf472922c0962644ef96109c6b204aba244ac68,0x5baf36801788b2d23d99b0797cf472922c0962644ef96109c6b204aba244ac68,26873570767257920454,15000000000000000,0xd77be88ba691a3e170a4d4b1643f4387b7d98b7c3d71706951b90e5db37e7954424a46612794dbb8000194937834250dc723ffa5610ba248,0.010036,0.015000,26.873571,40.164107,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,WETH,18,DAI,18,2021-08-05 08:03:55
107549,1628151011,57333334166,1699,0x9cb8bb8c8227e893c3a5be72ffb093dc02c4ef154cdc24bfa9a90bc4f67b4734,0x9cb8bb8c8227e893c3a5be72ffb093dc02c4ef154cdc24bfa9a90bc4f67b4734,4262565514161118,2849,0xb69bc1be0b676c615f729d73a90cd111adab5e31d1c8dd64fecd5be146c7cb86424a46612794dbb8000194937834250dc723ffa5610ba3c1,0.004263,0.010530,11.413476,28.427503,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,GUSD,2,NaN,18,2021-08-05 08:10:11
79953,1628151263,46333333517,15247857004985886720,0x0143df0301e2b14420109dd63073450e71a1277f585d9680cedbe9a632860097,0x0143df0301e2b14420109dd63073450e71a1277f585d9680cedbe9a632860097,18683766405690006,65472865720793357605,0xb8d7f2b053ce3ce02468153cb2fd40c3c68cf8405e7f8aa6adb8eec6de5870678bc0fe84022542e1611ff252090028df741ad670610ba4ad,0.018684,0.024445,50.077756,65.472866,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,DAI,18,NaN,18,2021-08-05 08:14:23


### do transformations


In [9]:
# get name of each file in chainlink_prices folder without the .csv
files = [f.split('.')[0] for f in os.listdir('data/transformed/chainlink_prices')]

# sort files alphabetically
files.sort()

In [10]:
dfs_data_list = []

In [11]:
counter = 0

for name in files:
    print(f'Round {counter}')
    # load price data from chainlink_prices folder
    chain_df = pd.read_csv(f'data//transformed/chainlink_prices/{name}.csv')

    # convert chain_df_sorted to datetime
    chain_df['date'] = pd.to_datetime(chain_df['timestamp'], unit='s')
    
    # sort by timestamp
    chain_df_sorted = chain_df.sort_values(by='timestamp')


    # check which rows of buyTokenSymbol and sellTokenSymbol are COMP
    cow_buy = cow_df[cow_df['buyTokenSymbol'] == name] # mask filter
    cow_sell = cow_df[cow_df['sellTokenSymbol'] == name] # mask filter

    # sort cow_buy by timestamp and plot
    cow_buy_sorted = cow_buy.sort_values(by='date')
    cow_sell_sorted = cow_sell.sort_values(by='date')

    # left inner join chaink_df with cow_buy on date. Interpolate if date does not match up
    cow_chain_buy = pd.merge_asof(cow_buy_sorted, chain_df_sorted, on='date', direction='nearest', tolerance=pd.Timedelta('1d'), allow_exact_matches=False)
    # left inner join chaink_df with cow_sell on date. Interpolate if date does not match up
    cow_chain_sell = pd.merge_asof(cow_sell_sorted, chain_df_sorted, on='date', direction='nearest', tolerance=pd.Timedelta('1d'), allow_exact_matches=False)

    dfs_data_list.append(cow_chain_buy)
    dfs_data_list.append(cow_chain_sell)

    counter += 1

Round 0
Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
Round 51
Round 52
Round 53
Round 54
Round 55
Round 56
Round 57
Round 58
Round 59
Round 60
Round 61
Round 62
Round 63
Round 64
Round 65
Round 66
Round 67
Round 68
Round 69
Round 70
Round 71
Round 72
Round 73
Round 74
Round 75
Round 76
Round 77
Round 78
Round 79
Round 80
Round 81
Round 82
Round 83
Round 84
Round 85
Round 86
Round 87
Round 88
Round 89
Round 90
Round 91
Round 92
Round 93
Round 94
Round 95
Round 96
Round 97
Round 98
Round 99
Round 100
Round 101
Round 102
Round 103
Round 104
Round 105
Round 106
Round 107


In [18]:
# combine all dataframes in dfs_data_list into one dataframe
cow_chain_df = pd.concat(dfs_data_list)

In [19]:
eth_chain_df = pd.read_csv(f'data//transformed/chainlink_prices/ETH.csv')

In [20]:
# minimize columns
eth_chain_df = eth_chain_df[['date', 'timestamp', 'price']]

# convert date column to datetime
eth_chain_df['date'] = pd.to_datetime(eth_chain_df['date'])

# sort by date
eth_chain_df = eth_chain_df.sort_values(by='date')

In [22]:
# sort cow_chain_df by date
cow_chain_df = cow_chain_df.sort_values(by='date')

In [23]:
# add eth buy and sell price
cow_chain_df = pd.merge_asof(cow_chain_df, eth_chain_df, on='date', direction='nearest', tolerance=pd.Timedelta('1d'), allow_exact_matches=False)

In [26]:
cow_chain_df

,timestamp_x,gasPrice,feeAmount,txHash,settlement,buyAmount,sellAmount,order,buyAmountEth,sellAmountEth,buyAmountUsd,sellAmountUsd,solvers,environment,name,active,sellTokenSymbol,sellTokenDecimals,buyTokenSymbol,buyTokenDecimals,date,id,assetPair_id,timestamp_y,price_x,timestamp,price_y
0,1628089484,55000000474,6776000000000000,0x08100e7ba81be84ee0bdce43db6640e2f992ec9991a740a689e97d20dea9dafa,0x08100e7ba81be84ee0bdce43db6640e2f992ec9991a740a689e97d20dea9dafa,1225092772597923267,100000000000000000,0x81dbb9631c0bec7d9762e170ff5feb2c9b366d5ca9502036d20d99c91cafb8bfb54872859733a3dfbb2c5401ac68cd9ca84b3cd1610ab36f,0.093887,0.100000,246.157990,264.102661,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,WETH,18,GNO,18,2021-08-04 15:04:44,ETH/USD/0x610aad51,WETH/USD,1.628090e+09,2656.010000,1628089681,2656.010000
1,1628146744,44000000000,3663931816677019648,0x8772387c9e80b23080cba1ce607defc91f6680d5803f745dedbcc218e98562e8,0x8772387c9e80b23080cba1ce607defc91f6680d5803f745dedbcc218e98562e8,17868105041905652,17060511063645525663,0xee0db370db55b657a0a2e507249d6e17575196c82434dee380fc1922e94b37967b2e78d4dfaaba045a167a70da285e30e8fca196610b92dc,0.017868,0.022882,47.899028,61.470674,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,BNT,18,WETH,18,2021-08-05 06:59:04,BNT/USD/0x610b1824,BNT/USD,1.628117e+09,3.642905,1628146521,2680.530000
2,1628146744,44000000000,3663931816677019648,0x8772387c9e80b23080cba1ce607defc91f6680d5803f745dedbcc218e98562e8,0x8772387c9e80b23080cba1ce607defc91f6680d5803f745dedbcc218e98562e8,17868105041905652,17060511063645525663,0xee0db370db55b657a0a2e507249d6e17575196c82434dee380fc1922e94b37967b2e78d4dfaaba045a167a70da285e30e8fca196610b92dc,0.017868,0.022882,47.899028,61.470674,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,BNT,18,WETH,18,2021-08-05 06:59:04,ETH/USD/0x610b8b59,WETH/USD,1.628147e+09,2680.530000,1628146521,2680.530000
3,1628150635,53250000505,5000000000000000,0x5baf36801788b2d23d99b0797cf472922c0962644ef96109c6b204aba244ac68,0x5baf36801788b2d23d99b0797cf472922c0962644ef96109c6b204aba244ac68,26873570767257920454,15000000000000000,0xd77be88ba691a3e170a4d4b1643f4387b7d98b7c3d71706951b90e5db37e7954424a46612794dbb8000194937834250dc723ffa5610ba248,0.010036,0.015000,26.873571,40.164107,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,WETH,18,DAI,18,2021-08-05 08:03:55,DAI/USD/0x610b990d,DAI/USD,1.628150e+09,1.000263,1628150483,2680.496318
4,1628150635,53250000505,5000000000000000,0x5baf36801788b2d23d99b0797cf472922c0962644ef96109c6b204aba244ac68,0x5baf36801788b2d23d99b0797cf472922c0962644ef96109c6b204aba244ac68,26873570767257920454,15000000000000000,0xd77be88ba691a3e170a4d4b1643f4387b7d98b7c3d71706951b90e5db37e7954424a46612794dbb8000194937834250dc723ffa5610ba248,0.010036,0.015000,26.873571,40.164107,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,WETH,18,DAI,18,2021-08-05 08:03:55,ETH/USD/0x610b9ad3,WETH/USD,1.628150e+09,2680.496318,1628150483,2680.496318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386657,1672419383,18002121440,35301,0xcea7ce585cd3faf72333250f7a9356b635eabe597d1e6b5eb3d4143b70ccfe46,0xcea7ce585cd3faf72333250f7a9356b635eabe597d1e6b5eb3d4143b70ccfe46,1763836245,10734800,0x2c6fa15f78a1e759118ba7dafbc6126f85237dcd8f2a7282b4f2ecc94b6b90edb0f76a4c60c6c993ac30bdda24c5f7f98a03249c63af1ed2,1.476822,1.482584,1763.836245,1770.717796,0x97ec0a17432d71a3234ef7173c6b48a2c0940896,prod,Quasilabs,True,WBTC,8,USDT,6,2022-12-30 16:56:23,USDT/USD/0x63af421f,USDT/USD,1.672430e+09,0.999500,1672419323,1194.280000
386658,1672420883,24315350224,1783651287793523712,0xf96021e612d0e34fe22dc765b02161158fe5e03c440e1d9aa308cc280148f53e,0xf96021e612d0e34fe22dc765b02161158fe5e03c440e1d9aa308cc280148f53e,2019470587060724849398,400000000000000000000,0xdaa1a237eede65bf8129806c4c8c91180fa9a28ed66cf7802aaa1a376a5d364506263e1a856b36e073ba7a50d24012341150161163af24ee,1.690859,1.701028,2019.470587,2031.615

In [27]:
#rename timestamp to eth_chain_timestamp, price_y to eth_chain_price
cow_chain_df = cow_chain_df.rename(columns={'timestamp_y': 'eth_chain_timestamp', 'price_y': 'eth_chain_price'})

In [28]:
# save cow_chain_df to csv
cow_chain_df.to_csv('data/transformed/cow_chain_swaps.csv', index=False)